__Corresponding Teacher__: Dawood AL CHANTI, MCF, PHELMA.

__Email__: dawood.al-chanti@grenoble-inp.fr


<center><b><font style="color: orange" size="5">BE: 5: Geometrical Transformation: Forward and Backward Mapping</font></b></center>
&ensp;


> <left><b><font style="color: blue" size="5"> Deadline: </font></b></center> &ensp;  <left><b><font style="color: red" size="5"> **31/03/2025 latest at 23h59**</font></b></center> &ensp;

> <left><b><font style="color: blue" size="3"> Student Information: </font></b></center> &ensp;


* Student 1:
    - Last Name: DUPAYAGE
    - First Name: Quentin
    - Identifier/algan: dupayagq
    
* Student 2:
    - Last Name: MELKIOR
    - First Name: Clément
    - Identifier/algan: melkiorc

> <left><b><font style="color: blue" size="3"> Working in pairs or alone. </font></b></center> &ensp;


* **To upload your work, follow the following steps:**
    - Go to https://chamilo.grenoble-inp.fr/ 
    - Go to the course `4PMSTIA5 Traitement d'images avancé`
        - Go to the section Travaux d'étudiants
        - Upload your work under `BE: séance 3`
        - **You must upload this `.ipynb` as it is, which will contain your code and your comments/analysis.**
            - **Any other formate, will not be corrected.**
            
            

> <left><b><font style="color: red" size="3">Important: </font></b></center> &ensp; <center><b><font style="color: red" size="3"> 
    
- **Consider delievering a clean notebook, that contain the meaningful experimental results and analysis. Remove other experiments that you performed, which you considered as just trial.**
 


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### Quantification metrics

In [ ]:
from skimage.metrics import structural_similarity as ssim
from math import log10
 
    
def mse(img1, img2):
    return np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)

def psnr(img1, img2):
    mse_val = mse(img1, img2)
    return 20 * log10(255.0 / np.sqrt(mse_val))
 

# IMPORTANT TO READ TO UNDERSTAND THE CONCEPT.

# Understanding Image Coordinates vs Cartesian Coordinates

When working with images in computer vision, it's important to understand the **difference between image coordinates and Cartesian coordinates** (centered at origin).

### Image Dimensions
We are working with an image of:

- **Width = 356 pixels**
- **Height = 256 pixels**

In **image coordinates**:
- The origin `(0, 0)` is at the **top-left corner**
- The x-axis goes **right**
- The y-axis goes **down**

In **Cartesian coordinates (centered)**:
- The origin `(0, 0)` is at the **center of the image**
- The x-axis goes **right**
- The y-axis goes **up**

---

### Mapping Image to Cartesian

To convert a pixel from image coordinates `(x_img, y_img)` to centered Cartesian coordinates `(x_cart, y_cart)`, we use:

```python
x_cart = x_img - (width / 2)
y_cart = y_img - (height / 2)


* So the top-left corner (0, 0) in image coordinates becomes:
 

```python
x_cart = 0 - 178 = -178
y_cart = 0 - 128 = -128


### Relation with geometrical transoformation:
* When performing geometric transformations (like rotation or scaling), we typically want to rotate or transform around the center of the image, not around the corner.

* By shifting the origin to the center, we ensure:
   - Rotations are centered
   - Transformations behave intuitively
   - Math is consistent with standard Cartesian geometry

In [ ]:
# Image dimensions
height = 256
width = 356

# Image center in image coordinates
x_center_img = width / 2
y_center_img = height / 2

# Top-left corner in Cartesian (centered) coords
top_left_cartesian_x = 0 - x_center_img  # -178
top_left_cartesian_y = 0 - y_center_img  # -128

# Create a canvas for display
fig, ax = plt.subplots(figsize=(8, 9))
ax.set_xlim(-width / 2 - 50, width / 2 + 50)
ax.set_ylim(-height / 2 - 50, height / 2 + 50)
ax.set_aspect('equal')

# Draw full image rectangle centered at (0, 0)
rect = plt.Rectangle((-width/2, -height/2), width, height,
                     edgecolor='blue', facecolor='none', linewidth=2)
ax.add_patch(rect)

# Mark origin
ax.plot(0, 0, 'ro')
ax.text(-15, -10, 'Origin (0, 0)', color='red', fontsize=12, fontweight='bold')

# Mark top-left corner
ax.plot(top_left_cartesian_x, top_left_cartesian_y, 'go')
ax.text(top_left_cartesian_x -10, top_left_cartesian_y -10,
        f'Top-Left (Image: 0,0)\nCartesian: ({int(top_left_cartesian_x)}, {int(top_left_cartesian_y)})',
        color='green', fontsize=11)

# Draw arrows for X and Y axes to represent directions
arrow_len = 60
ax.arrow(0, 0, arrow_len, 0, head_width=10, head_length=10, fc='black', ec='black')
ax.arrow(0, 0, 0, arrow_len, head_width=10, head_length=10, fc='black', ec='black')

# Labels for directions (shifted for clarity)
ax.text(arrow_len + 15, 5, 'X →', fontsize=12, color='black', verticalalignment='bottom')
ax.text(5, arrow_len + 15, 'Y ↓', fontsize=12, color='black', horizontalalignment='left')

# Flip Y-axis to match image coordinate system (top-left origin)
ax.invert_yaxis()

# Axes and labels
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=0.5)
ax.set_title("Image Coordinate (0,0) Mapped to Centered Cartesian Domain", fontsize=14)
ax.set_xlabel("X (centered)")
ax.set_ylabel("Y (centered)")
ax.grid(True)

plt.tight_layout()
plt.show()


-------------------

# Session starts here:

### Load and Display Image called `parrot.jpg`

In [ ]:
# Load grayscale image
image = cv2.imread("parrot.jpg", cv2.IMREAD_GRAYSCALE)

# Get image dimensions
rows, cols = image.shape

# Display original image
plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray')
plt.title("Affichage des lignes de l'image")
#plt.legend()
plt.show()
    

--------------------

-------------------

### Load and Display ground truth (transformed image), it is saved as `ground_truth.npy'
   * You will use this image as your ground truth to quantify and analyse the results.

In [ ]:
# Load the saved .npy file fot that use np.load
loaded_gt = np.load("ground_truth.npy")

In [ ]:
# Display Ground Truth Image image
plt.figure(figsize=(8, 8))
plt.imshow(loaded_gt, cmap='gray')
plt.title("Affichage des lignes de l'image")
#plt.legend()
plt.show()
    
    
    

### Define a   transformation matrix `T`  based on the following transformation parameters:

   * `angle_deg` which define the amount of rotaion.
   * `tx` which define the amount of translation in `x` direction.
   * `ty` which define the amount of translation in `y` direction

In [ ]:
angle_deg = 30.6        
tx = 22.3               
ty = 10.15             

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)


#Build 3x3 affine transformation matrix T  in homogenous domain (clockwise)
T = np.array([
    [cos_a, -sin_a, tx],
    [sin_a, cos_a, ty],
    [0, 0, 1]
])

## Q. Can you explain what this transformation does to the spatial structure of the image?

Ca tourne l'image dans le sens horaire de 30.6 ° et la translate sur l'axe x de Dx = 22.3 pixels, sur l'axe y de Dy = 10.15 pixels

# Experiment 1: Forward Mapping with NN interpolation

## Perform Forward Mapping with Nearest Neighbor Interpolation. To do that, rely on the pseudo code given below.

### Pseudocode: Forward Mapping with NN Interpolation.

Input: 
   - image: 2D grayscale image
   - T: 3x3 transformation matrix (rotation + translation)

Output:
   - `output_forward_nn`: transformed image.

1. Get the number of rows and columns in the image

2. Create a new output image filled with zeros (same size as input) call it `output_forward_nn`

3. For each pixel (x, y) in the input image:
    a. Subtract image center to get centered coordinates:
        x_centered = ###
        y_centered = ###

    b. Apply the transformation T to the centered coordinates:
        new_x = ### * ### + ### * ### + ###
        new_y = ### * ### + ### * ### + ###

    c. Shift the new coordinates back to image space:
        new_x = ### + ###
        new_y = ### + ###

    d. Round new_x and new_y to the nearest integers simply using ``int(round(value))``
        new_x_int = ###
        new_y_int = ###

    e. If the new coordinates `(new_x_int,new_y_int)` are inside the image bounds `(0 <= new_x_int < cols` and `0 <= new_y_int < rows)`: 
        - Copy the pixel value from (x, y) in the original image to (new_x_int, new_y_int) in the output image `output_forward_nn`


### Display the output image and the ground truth

In [ ]:
def out_forward_nn (image, T):
    """
    Transformation method in order to apply structural transformation on an image

    Parameters:
    - image: Input image (NumPy array)
    - T: Matrix of the structural transformation

    Returns:
    - Transformed image
    """
    # 1. Récupérer le nombre de ligne et de colonne, ainsi que les centres
    rows, cols = image.shape
    center_x = cols / 2
    center_y = rows / 2

    # 2. Créer l'image de sortie 
    output_forward_nn = np.zeros_like(image)
    
    # 3. Parcourir chaques pixels de l'image de départ
    for y in range(rows):
        for x in range(cols):
            # a. Coordonées centrées
            x_centered = x - center_x
            y_centered = y - center_y

            # b. Appliquer la transformation
            new_x = T[0, 0] * x_centered + T[0, 1] * y_centered + T[0, 2]
            new_y = T[1, 0] * x_centered + T[1, 1] * y_centered + T[1, 2]

            # c. Revenir à l'espace image
            new_x += center_x
            new_y += center_y

            # d. Arrondir aux entiers les plus proches
            new_x_int = int(round(new_x))
            new_y_int = int(round(new_y))

            # e. Vérifier si les nouvelles coordonnées sont dans les limites
            if 0 <= new_x_int < cols and 0 <= new_y_int < rows:
                output_forward_nn[new_y_int, new_x_int] = image[y, x]

    return output_forward_nn
    
# Apply the transformation on the image
output_forward_nn = out_forward_nn(image, T)

fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Show the forward mapped image
axes[0].imshow(output_forward_nn, cmap='gray')

# Show the loaded ground truth image
axes[1].imshow(loaded_gt, cmap='gray')

Nous remarquons que l'image reconstruite n'est pas nette, il y a quelques aspéritées: Lors de la construction de notre image transformée, nous partons d'une matrice "np.zeros_like(image)" (étape 2). Lors de l'attribution de  nos "new_x_int" et "new_y_int", nous appliquons l'arrondis ainsi que la conversion en entier. Ainsi, certaines positions de pixels ne sont pas atteintes, et par conséquent pas attribuées, elles restent par conséquent à 0.

# Quantifying the results

You are provided with a set of quantitative metrics to help evaluate the difference between the ground truth and the transformed image you obtained.
---

### 1. **MSE (Mean Squared Error)** 
The average squared difference between each pixel in the transformed image and the corresponding pixel in the ground truth image.

**Formula:**

\\[
\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} \left( I_{\text{gt}}[i] - I_{\text{output}}[i] \right)^2
\\]
- Lower MSE = better quality
- Sensitive to large pixel differences

---

### 2. **PSNR (Peak Signal-to-Noise Ratio)**
A logarithmic measurement of the peak possible pixel value vs. the observed error (MSE).

**Formula:**

\\[
\text{PSNR} = 20 \cdot \log_{10} \left( \frac{\text{MAX}_I}{\sqrt{\text{MSE}}} \right)
\\]

Where \\(\text{MAX}_I = 255\\) for 8-bit grayscale images.
- Higher PSNR = better image quality
---

### 3. **SSIM (Structural Similarity Index)**
SSIM compares luminance, contrast, and structure between the two images — closer to how humans perceive visual quality.

**Range:**  
\\[
0 \leq \text{SSIM} \leq 1
\\]

- SSIM closer to 1 = higher structural similarity


### Compute the mean square error using the provided function `mse()`, Peak Signal-to-Noise Ratio using `psnr()` and Structural Similarity Index ssim()


  

In [ ]:
mse_valFNN = mse(output_forward_nn, loaded_gt)
psnr_valFNN = psnr(output_forward_nn, loaded_gt)
ssim_valFNN = ssim(output_forward_nn, loaded_gt)


#Given:
print(f"{'Method':<20} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 70)
print(f"{'Forward NN':<20} | {mse_valFNN:10.2f} | {psnr_valFNN:10.2f} | {ssim_valFNN:10.4f}")

## Quantitative Result Analysis

After applying **Forward Mapping (Nearest Neighbor)**, you obtained the evaluation metrics by comparing your output with the ground truth image:

### Questions for Analysis

1. **Mean Squared Error (MSE)**  
   - What does MSE measure in this context?
   - What could cause the MSE to be higher?

2. **Peak Signal-to-Noise Ratio (PSNR)**  
   - What does PSNR tell you about image quality?
   - Generally, PSNR > 30 dB is considered high quality. How does your result compare?
   - What does a low PSNR may indicate in image transformation tasks?

3. **Structural Similarity Index (SSIM)**  
   - SSIM ranges from 0 to 1. What does your SSIM value indicate?
   - Does it reflect high structural similarity to the original image?

---

###  Reflection Analysis
- Forward mapping can leave holes or unassigned pixels in the output image as you observed.
  - How could this affect the MSE and SSIM?
  - Why is backward mapping often preferred in practical applications?


MSE = mean square error: mesure la moyenne de la différence au carré entre chaques pixels de notre image transformée avec la ground truth image. Ainsi, plus il y aura de différence entre chaques pixels des deux images, au plus notre MSE sera important (ici 2015.57 est un gros MSE, résultant du fait que de nombreux pixels sont à 0)
PSNR = Peak Signal-to-Noise Ratio: Mesure logarithmique du rapport entre l'intensité maximal de l'image et la racine carré du MSE de l'image. Ainsi, deux facteurs rentrent en jeux pour cette mesure: avoir une importante luminosité dans l'image (pour une image en niveau de gris, max_I = 255), ainsi qu'un MSE faible relativement faible par rapport à l'intensité de l'image. Nous pouvons remarquer qu'au plus une image aura une intensité lumineuse importante, au plus Max_I sera important, mais au plus le MSE sera important s'il y a des "dead-pixels" (pixels à 0), ainsi ces deux facteurs sont tout de même relié. Un PSNR > 30 indique que nous devons avoir un rapport (Max_I/sqrt(MSE)) minimum de 4.48, ce qui n'est pas notre cas, nous obtenons un PSNR de 15.09 due à notre MSE important par rapport à l'intensité générale de l'image. Cela indique que nous avons trop de dead-pixels dans notre image transformée.
SSIM = Structural Similitude Index: Notre SSIM est relativement bas (0.2883) due surtout au contraste et la luminosité entre les 2 images. Les trou (dead-pixels) créaient trop de constraste entre les deux image, baissant la luminosité générale de l'image et ainsi la similitude détecté entre les structures

Comme expliqué précédemment, la création de "dead-pixels" dans l'image va engendré de plus grosse erreur sur certains pixels, ce qui va énormément faire croître la MSE, ainsi que la contraste entre les deux image, ce qui affecte énormément la MSE (MSE va être plus importante) ainsi que la SSIM (SSIM va être plus proche de 0).
L'avantage du backward mapping est que l'on ne créer pas de trou: pour chaques pixels de l'image de départ (loaded_gt), nous attribuons un pixel de l'image d'arrivée (image). Chaques pixels a alors une valeur non-nul.

In [ ]:
#Build 3x3 affine transformation matrix T2 in homogenous domain (counter clockwise)
T2 = np.array([
    [cos_a, sin_a, tx],
    [-sin_a, cos_a, ty],
    [0, 0, 1]
])

def out_backward_nn(image, T):
    """
    Transformation method in order to apply structural transformation on an image

    Parameters:
    - image: Input image (NumPy array)
    - T: Matrix of the structural transformation

    Returns:
    - Transformed image
    """
    # 1. Récupérer les lignes et les colonnes, calculer les centres
    rows, cols = image.shape
    center_x, center_y = cols / 2, rows / 2

    # 2. Créer l'image et calcul l'inverse de la transformation
    output_image = np.zeros_like(image)
    T_inv = np.linalg.inv(T)

    # 3. Pour chaques pixels
    for y_out in range(rows):
        for x_out in range(cols):
            # a. Coordonées centrées de l'image de sortie
            x_centered_out = x_out - center_x
            y_centered_out = y_out - center_y

            # b. Appliquer la transformation inverse
            src_x = T_inv[0,0]*x_centered_out + T_inv[0,1]*y_centered_out + T_inv[0,2]
            src_y = T_inv[1,0]*x_centered_out + T_inv[1,1]*y_centered_out + T_inv[1,2]
            x_in, y_in = src_x + center_x, src_y + center_y

            # c. Arrondir pour obtenir le plus proche voisin
            x_nearest = int(round(x_in))
            y_nearest = int(round(y_in))

            # d. Vérifier si on est dans les bornes de l'image d'origine
            if 0 <= x_nearest < cols and 0 <= y_nearest < rows:
                output_image[y_out, x_out] = image[y_nearest, x_nearest]

    return output_image

In [ ]:
# Apply the transformation on the image
output_backward_nn = out_backward_nn(loaded_gt, T2)

fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Show the backward mapped loaded ground truth image
axes[0].imshow(output_backward_nn, cmap='gray')

# Show the original image
axes[1].imshow(image, cmap='gray')

Nous voyons que l'image est bien plus nette, et qu'il n'y a pas de trou.
(Je n'avais pas vu que nous allions l'implémenter plus bas, je voulais simplement visualiser la différence)


# Experiment 2: Forward Mapping with Bilinear Interpolation.


In [ ]:
from scipy.ndimage import map_coordinates


#### Forward Mapping Pseudocode with Bilinear Interpolation.
* Here we will use the function `map_coordinates()` to perform the bilinear interpolation as it is optimized and fast. However, if you are interested in details, refer to the course slide and here is the related python code :

```python
if 0 <= new_x < cols - 1 and 0 <= new_y < rows - 1:
    x0 = int(np.floor(new_x))
    y0 = int(np.floor(new_y))
    x1 = x0 + 1
    y1 = y0 + 1

    # Compute weights
    dx = new_x - x0
    dy = new_y - y0

    # Pixel contributions
    w00 = (1 - dx) * (1 - dy)
    w01 = dx * (1 - dy)
    w10 = (1 - dx) * dy
    w11 = dx * dy

    # Distribute intensity using bilinear weights
    output_forward_nn[y0, x0] += w00 * image[y, x]
    output_forward_nn[y0, x1] += w01 * image[y, x]
    output_forward_nn[y1, x0] += w10 * image[y, x]
    output_forward_nn[y1, x1] += w11 * image[y, x]


## Pseudocode to implement:

Input: 
   - image: 2D grayscale image
   - T: 3x3 transformation matrix (rotation + translation)

Output:
   - `output_forward_nn_bilinear`: transformed image.

1. Get the number of rows and columns in the image

2. Create a new output image filled with zeros (same size as input) call it `output_forward_nn_bilinear`

3. For each pixel (x, y) in the input image:
    a. Subtract image center to get centered coordinates:
        x_centered = ###
        y_centered = ###

    b. Apply the transformation T to the centered coordinates:
        new_x = ### * ### + ### * ### + ###
        new_y = ### * ### + ### * ### + ###

    c. Shift the new coordinates back to image space:
        new_x = ### + ###
        new_y = ### + ###

    d. If the new coordinates `(new_x,new_y)` are inside the image bounds `(0 <= new_x < cols` and `0 <= new_y < rows)`: 
      
            Use scipy to interpolate at subpixel position
            interpolated_value = map_coordinates(
                image,
                [[###], [###]],      # Coordinates in (row, col) order
                order=1              # 1 = bilinear, 3 = cubic spline
            )[0]

            output_forward_nn_bilinear[###, ###] = ###

        

In [ ]:
angle_deg = -30.6        
tx = -22.3               
ty = -10.15             

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)


#Build 3x3 affine transformation matrix T  in homogenous domain (clockwise)
T = np.array([
    [cos_a, -sin_a, tx],
    [sin_a, cos_a, ty],
    [0, 0, 1]
])

In [ ]:
def forward_bilinear (image, T, order = 1):
    """
    Transformation method in order to apply structural transformation on an image

    Parameters:
    - image: Input image (NumPy array)
    - T: Matrix of the structural transformation
    - order: Integer, parameter of map_coordinates function, default = 1

    Returns:
    - Transformed image
    """
    # 1. Récupérer le nombre de ligne et de colonne, ainsi que les centres
    rows, cols = image.shape
    center_x = cols / 2
    center_y = rows / 2

    # 2. Créer l'image de sortie 
    output_forward_bilinear = np.zeros_like(image)
    
    # 3. Parcourir chaques pixels de l'image de départ
    for y in range(rows):
        for x in range(cols):
            # a. Coordonées centrées
            x_centered = x - center_x
            y_centered = y - center_y

            # b. Appliquer la transformation
            new_x = T[0, 0] * x_centered + T[0, 1] * y_centered + T[0, 2]
            new_y = T[1, 0] * x_centered + T[1, 1] * y_centered + T[1, 2]

            # c. Revenir à l'espace image
            new_x += center_x
            new_y += center_y

            # d. Vérifier si les nouvelles coordonnées sont dans les limites
            if 0 <= new_x < cols and 0 <= new_y < rows:
                interpolated_value = map_coordinates(
                image,
                [[new_y], [new_x]],      # Coordinates in (row, col) order
                order= order              # 1 = bilinear, 3 = cubic spline
                )[0]
                output_forward_bilinear[y,x] = interpolated_value
    return output_forward_bilinear

La méthode cubic spline nécessite trop de temps de calculs pour être réaliser (sans y passer 1 h) et nous n'obtiendrions pas de tellement meilleurs résultats: l'image serait encore plus flouté.

In [ ]:
# Forward method with bilinear interpolation and order = 1
output_forward_bilinear = forward_bilinear(image, T)

fig, axes = plt.subplots(1, 3, figsize=(14, 8))

# Show the forward mapped image
axes[0].imshow(output_forward_nn, cmap='gray')
axes[0].set_title("Forward Mapping: Transformed Image (NN operator)")

# Show the forward mapped image
axes[1].imshow(output_forward_bilinear, cmap='gray')
axes[1].set_title("Forward Mapping: Transformed Image (Bilinear operator)")

# Show the loaded ground truth image
axes[2].imshow(loaded_gt, cmap='gray')
axes[2].set_title("Ground Truth Image")


plt.tight_layout()
plt.show()
    

mse_valFBiL = mse(output_forward_bilinear,loaded_gt)
psnr_valFBiL = psnr(output_forward_bilinear,loaded_gt)
ssim_valFBiL = ssim(output_forward_bilinear,loaded_gt)
    
#Given:    
print(f"{'Method':<20} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 65)
print(f"{'Forward bilinear':<20} | {mse_valFBiL:10.2f} | {psnr_valFBiL:10.2f} | {ssim_valFBiL:10.4f}")


## Experiment Analysis

As in the previous experiment, evaluate the results both **qualitatively** and **quantitatively**. Provide your observations on the output image in terms of:

- Visual quality  
- Structure preservation  
- Any visible distortions  

Then, compare these results with those obtained in the **first experiment**. Highlight key differences in terms of:

- Accuracy  
- Interpolation smoothness  
- Artifact presence  

---

### 1. Quantitative Evaluation

Compute and report the following metrics between the **original ground truth image** and the **transformed (mapped-back) result**:


- **MSE (Mean Squared Error)**  
- **PSNR (Peak Signal-to-Noise Ratio)**  
- **SSIM (Structural Similarity Index)**  

| Method                     | MSE       | PSNR (dB) | SSIM     |
|----------------------------|-----------|-----------|----------|
| Nearest Neighbor           | 2015.57   | 15.09     | 0.2883   |
| Bilinear Interpolation     | 1247.23   | 17.17     | 0.6183   |

> _Replace the placeholders with your actual computed results.

---

### 2. Qualitative Evaluation

#### Visual Inspection Checklist:

- **Sharpness** of details  
- Presence or absence of **aliasing or blocky artifacts**  
- How well **edges and fine structures** are preserved  
- Any **blurring** introduced due to interpolation  


### Observations:

- Summarize your findings here. Comment on which method is more suitable and in what context.


Pour l'évaluation quantitative, nous observons de bien meilleurs résultats avec l'interpolation bilinéaire (meilleur MSE, car plus bas, résultant sur un meilleur PSNR et surtout un bien meilleur indice de similarité de structure). L'interpolation par NN (Nearest Neighbor) créer des 'dead-pixels' qui augmente énormément le MSE, alors que l'interpolation bilinéaire attribut tous les pixels de départ, ce qui évite les trou, ainsi que les superpositions de pixels. Bien que l'image soit de bien meilleure qualité, nette, et sans aspéritées, elle n'a cependant pas un MSE nul: c'est due au fait que nous attribuons des valeurs à des pixels "flottants" (et non entier), et que l'opération de bilinéarité fait intervenir les valeurs pondérées des pixes voisins, ce qui modifie la valeur du pixel.

Pour l'évalutation qualitative, nous observons également de bien meilleurs résultats avec l'interpolation bilinéaire (MSE plus faible, meilleure correspondance structurelle). Comme dis précédemment, l'interpolation NN va alors créer les trou dans l'image, ce qui la rend beaucoup moins nette. Cependant, alors que l'interpolation NN ne modifie pas la valeur des pixels, l'interpolation bilinéaire introduit une pondération des pixels représentés par les 4 plus proches voisins, ce qui peut avoir un effet de "blurring" sur les pixels, notament visible sur le bord de l'image, où certains pixel perdent de leurs intensités due au bord noir. Une solution pour réduire cet effet serait d'utiliser "order=3" (ou cubic spline), qui prend en compte plus de voisins, et ainsi adoucie encore plus cet effet.

Ainsi, l'interpolation par NN permet de conserver les valeurs exactes des pixels, mais créée des trou (dead-pixels), alors que l'interpolation bilinéaire restitue une meilleure image au globale, mais modifie la valeur des pixels due à la pondération.

Ainsi, lorsque nous voudrons être plus précis sur l'image, avec une restitution nette de l'image (imagerie médicale, image satélite, cinéma), la méthode par interpolation bilinéaire sera préférable (meilleur MSE et surtout meilleur SSIM), et lorsque nous voudrons plus faire de la classification, nous utiliserons la méthode par interpolation NN afin de garder au maximum le placement des pixels ainsi que leurs valeurs réelles (et non pondérées)

# Experiment 3: backward Mapping with NN Interpolation.

### Backward Mapping With Nearest Neighbor Interpolation

# Reminder


* **Forward Mapping**: Map each input pixel to its destination in the output using the forward transformation.
* For each pixel in the input image:

   - Compute where it should go in the output.
   - Move that pixel to its new position.

```python
for each pixel (x, y) in input_image:
    (dst_x, dst_y) = forward_transform(x, y)
    output_image[dst_x, dst_y] = input_image[x, y]



* **Backward Mapping** (Inverse Mapping): Map each output pixel back to its source in the input image using the inverse transformation.
* For each pixel in the output image:
   - Compute where it came from in the input.
   - Copy that source pixel into the current output location.
   

```python
for each pixel (x, y) in output_image:
    (src_x, src_y) = inverse_transform(x, y)
    output_image[x, y] = input_image[src_x, src_y]


### Pseudocode: Backward Mapping with NN Interpolation.

Input: 
    - Original image
    - Inverse transformation matrix T_inv
    - Image dimensions: rows, cols

Output: 
    - Transformed output image `output_backward_nn`

Procedure:

1. Get the number of rows and columns in the output image.


2. Initialize an empty output image of the same size as the input  `output_backward_nn`.

3. For each pixel (x, y) in the output image:
    
    a. Center the pixel coordinates around the image center:
        x_centered = ###
        y_centered = ###

    b. Apply the inverse transformation to get source coordinates:
        src_x = ### * ### + ### * ### + ###
        src_y = ### * ### + ### * ### + ###

    c. Shift the source coordinates back to image space:
        src_x = ### + ###
        src_y = ### + ###

    d. Use nearest neighbor interpolation  simply using ``int(round(value))``:
        src_x_nn = ###
        src_y_nn = ###

    e. If (src_x_nn, src_y_nn) is inside image bounds `(0 <= src_x_nn < cols) and (0 <= src_y_nn < rows)`:
        Assign the pixel:
            output[y, x] = input[src_y_nn, src_x_nn]

4. Display or return the output image


### Given: here is the transofrmation parameters and the transformation matrix you will use.

In [ ]:
# Define transformation parameters
angle_deg = 30.6       # Rotation angle in degrees
tx = 22.3              # Translation along x
ty = 10.15             # Translation along y

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)

T = np.array([
    [cos_a, sin_a, tx],
    [-sin_a, cos_a, ty],
    [0,0,1]
]) 

T2 = np.array([
    [cos_a, sin_a, -tx],
    [-sin_a, cos_a, -ty],
    [0,0,1]
]) 

The function is already coded, i defined it in the experiment 1

In [ ]:
output_backward_nn = out_backward_nn(loaded_gt,T)
output_backward_nn_2 = out_backward_nn(loaded_gt,T2)

### Display the Ground truth `loaded_gt`, the `output_forward_nn` and  the new output `output_backward_nn` in a subplot. Observe and comment on the visual quanlity of these images.

In [ ]:
 
# Create a 1x5 grid of subplots
fig, axes = plt.subplots(1, 5, figsize=(15, 6))

# Forward Mapping
axes[0].imshow(output_forward_nn, cmap='gray')
axes[0].set_title("Output_forward_NN")
axes[0].axis("off")

# Ground Truth
axes[1].imshow(loaded_gt, cmap='gray')
axes[1].set_title("loaded_ground_truth")
axes[1].axis("off")

# Original Image
axes[2].imshow(image, cmap='gray')
axes[2].set_title("Original Image")
axes[2].axis("off")

# Backward Mapping
axes[3].imshow(output_backward_nn, cmap='gray')
axes[3].set_title("Output_backward_NN")
axes[3].axis("off")

# Backward Mapping (reverse translation)
axes[4].imshow(output_backward_nn_2, cmap='gray')
axes[4].set_title("Output_backward_NN with reverse translation")
axes[4].axis("off")
 
plt.tight_layout()
plt.show()




Comme nous le voyons, notre image "backward" ne présente aucunes aspérités, et représente très bien les structures

### Quantify the results of `output_backward_nn`` with respect to the ground truth image `image_gt` using the functions `mse`, `psnr`, and `ssim`

In [ ]:
# Measure with the loaded_gt
mse_valBNN = mse(output_backward_nn,loaded_gt)
psnr_valBNN = psnr(output_backward_nn,loaded_gt)
ssim_valBNN = ssim(output_backward_nn,loaded_gt)

# Measure with the original image
mse_valBNN_I = mse(output_backward_nn_2,loaded_gt)
psnr_valBNN_I = psnr(output_backward_nn_2,loaded_gt)
ssim_valBNN_I = ssim(output_backward_nn_2,loaded_gt)

print(f"{'Method':<21} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 65)
print(f"{'backward NN (tx/ty)':<21} | {mse_valBNN:10.2f} | {psnr_valBNN:10.2f} | {ssim_valBNN:10.4f}")
print("-" * 65)
print(f"{'backward NN (-tx/-ty)':<21} | {mse_valBNN_I:10.2f} | {psnr_valBNN_I:10.2f} | {ssim_valBNN_I:10.4f}")


Nous avons essayer de décaler de -tx/-ty car nous trouvions que l'image d'output était meilleure, mais pas d'après les mesures, nous l'avons laisser.

## Experiment Analysis

As in the previous experiment, evaluate the results both **qualitatively** and **quantitatively**. Provide your observations on the output image in terms of:

- Visual quality  
- Structure preservation  
- Any visible distortions  

Then, compare these results with those obtained in the **first and second experiment**. Highlight key differences in terms of:

- Accuracy  
- Interpolation smoothness  
- Artifact presence  

---

### 1. Quantitative Evaluation

Compute and report the following metrics between the **original ground truth image** and the **transformed (mapped-back) result**:


- **MSE (Mean Squared Error)**  
- **PSNR (Peak Signal-to-Noise Ratio)**  
- **SSIM (Structural Similarity Index)**  


> _Replace the placeholders with your actual computed results.

| Method                  | MSE       | PSNR (dB) | SSIM     |
|-------------------------|-----------|-----------|----------|
| Forward Mapping NN      | 2015.57   | 15.09     | 0.2883   |
| Forward Mapping Bilinear| 1247.23   | 17.17     | 0.6183   |
| Backward Mapping NN     | 6050.76   | 10.31     | 0.4077   |

---

### 2. Qualitative Evaluation

#### Visual Inspection Checklist:

- **Sharpness** of details  
- Presence or absence of **aliasing or blocky artifacts**  
- How well **edges and fine structures** are preserved  
- Any **blurring** introduced due to interpolation  


### Observations:

- Summarize your findings here. Comment on which method is more suitable and in what context.


Du fait que de nombreux pixels sont noir (due à la perte de d'information des coins de l'image 'loaded_gt'), le MSE (influant sur le PSNR) est très élevé. Pour rappel, le MSE mesurant, pour chaques pixels, la différence entre les deux images, va alors énormément croître. Malgrès celà, l'indice de similitude (SSIM) est supérieur à la méthode de forward mapping avec l'interpolation NN: en effet, comme l'image est nette et que tous les pixels sont attribués, la similitude entre les deux objets est plus importante que l'image contenant des "trou".

# Experiment 4: backward Mapping with Bilinear Interpolation.

### Modify the pseudocode implementation of experiment 3 in `Backward Mapping with NN Interpolation` to perform  Backward Mapping with bilinear Interpolation  using the function `map_coordinates()`

In [ ]:
import numpy as np
from scipy.ndimage import map_coordinates



# Define transformation parameters
angle_deg = -30.6       # Rotation angle in degrees
tx = 22.3              # Translation along x
ty = 10.15             # Translation along y

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)

T = np.array([
    [cos_a, -sin_a, tx],
    [sin_a, cos_a, ty],
    [0,0,1]
]) 


In [ ]:
def out_backward_bilinear(image, T, order =1):
    """
    Transformation method in order to apply structural transformation on an image

    Parameters:
    - image: Input image (NumPy array)
    - T: Matrix of the structural transformation

    Returns:
    - Transformed image
    """
    # 1. Récupérer les lignes et les colonnes, calculer les centres
    rows, cols = image.shape
    center_x, center_y = cols / 2, rows / 2

    # 2. Créer l'image et calcul l'inverse de la transformation
    output_image = np.zeros_like(image)
    T_inv = np.linalg.inv(T)

    # 3. Pour chaques pixels
    for y_out in range(rows):
        for x_out in range(cols):
            # a. Coordonées centrées de l'image de sortie
            x_centered_out = x_out - center_x
            y_centered_out = y_out - center_y

            # b. Appliquer la transformation inverse
            src_x = T_inv[0,0]*x_centered_out + T_inv[0,1]*y_centered_out + T_inv[0,2]
            src_y = T_inv[1,0]*x_centered_out + T_inv[1,1]*y_centered_out + T_inv[1,2]

            # c. Revenir à l'espace image
            x_in, y_in = src_x + center_x, src_y + center_y

            # d. Vérifier si les nouvelles coordonnées sont dans les limites
            if 0 <= x_in < cols and 0 <= y_in < rows:
                interpolated_value = map_coordinates(
                image,
                [[y_in], [x_in]],      # Coordinates in (row, col) order
                order= order              # 1 = bilinear, 3 = cubic spline
                )[0]
                output_image[y_out,x_out] = interpolated_value

    return output_image

In [ ]:
output_backward_bilinear = out_backward_bilinear(loaded_gt,T)

# Create a 1x3 grid of subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# Backward (NN) Mapping
axes[0].imshow(output_backward_nn, cmap='gray')
axes[0].set_title("Output_backward_NN")
axes[0].axis("off")

# Ground Truth
axes[1].imshow(loaded_gt, cmap='gray')
axes[1].set_title("loaded_ground_truth")
axes[1].axis("off")

# Backward (bilinear) Mapping
axes[2].imshow(output_backward_bilinear, cmap='gray')
axes[2].set_title("Output_backward_Bil")
axes[2].axis("off")

In [ ]:
mse_valBBiL = mse(output_backward_bilinear, loaded_gt)
psnr_valBBiL = psnr(output_backward_bilinear, loaded_gt)
ssim_valBBiL = ssim(output_backward_bilinear, loaded_gt)

    
#Given
print(f"{'Method':<20} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 50)
print(f"{'backward NN':<20} | {mse_valBBiL:10.2f} | {psnr_valBBiL:10.2f} | {ssim_valBBiL:10.4f}")


In [ ]:

#Given: Metrics for all experiments
print(f"{'Method':<20} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 50)
print(f"{'Forward NN':<20} | {mse_valFNN:10.2f} | {psnr_valFNN:10.2f} | {ssim_valFNN:10.4f}")
print(f"{'Forward BiL':<20} | {mse_valFBiL:10.2f} | {psnr_valFBiL:10.2f} | {ssim_valFBiL:10.4f}")
print(f"{'backward NN':<20} | {mse_valBNN:10.2f} | {psnr_valBNN:10.2f} | {ssim_valBNN:10.4f}")
print(f"{'backward BiL':<20} | {mse_valBBiL:10.2f} | {psnr_valBBiL:10.2f} | {ssim_valBBiL:10.4f}")


## Experiment Analysis

As in the previous experiment, evaluate the results both **qualitatively** and **quantitatively**. Provide your observations on the output image in terms of:

- Visual quality  
- Structure preservation  
- Any visible distortions  

Then, compare these results with those obtained in the **first, second and third experiment**. Highlight key differences in terms of:

- Accuracy  
- Interpolation smoothness  
- Artifact presence  

---

### 1. Quantitative Evaluation

Compute and report the following metrics between the **original ground truth image** and the **transformed (mapped-back) result**:

- **MSE (Mean Squared Error)**  
- **PSNR (Peak Signal-to-Noise Ratio)**  
- **SSIM (Structural Similarity Index)**  


> _Replace the placeholders with your actual computed results.

| Method                  | MSE       | PSNR (dB) | SSIM     |
|-------------------------|-----------|-----------|----------|
| Forward Mapping NN      | 2015.57   | 15.09     | 0.2883   |
| Forward Mapping Bilinear| 1247.23   | 17.17     | 0.6183   |
| Backward Mapping NN     | 6050.76   | 10.31     | 0.4077   |
| Backward Mapping BiL    | 6024.56   | 10.33     | 0.4149   |

---

### 2. Qualitative Evaluation

#### Visual Inspection Checklist:

- **Sharpness** of details  
- Presence or absence of **aliasing or blocky artifacts**  
- How well **edges and fine structures** are preserved  
- Any **blurring** introduced due to interpolation  


### Observations:

- Summarize your findings here. Comment on which method is more suitable and in what context.


On observe que l'utilisation de l'interpolation bilinéaire n'affecte pas beaucoup les mesures, étant donnée qu'elle ne peut pas récupérer les 'dead-pixels". On observe tout de même une légère amélioration, que ce soit sur le MSE ou sur le SSIM: l'amélioration du MSE est surement due au fait que les pixels sur le bord (initialement noir) vont être pondérés par les pixels ayant une intensité non-nul, réduisant ainsi un peu l'erreur. Ce qui change également un peu le contraste et donc améliore un peu le SSIM. 

# Optional

# Experiment 5: Bidirectional mapping: (A → B and B → A),

1. Choose the algo developed in Experiment 4.
2. Perform mapping from the image to the transoformation.
3. Perform mapping from the transformation back to the image.
4. Perform qualitative and quantitative evaluation, here your ground truth image changed, it is the original image itself.

* **Note:** Measuring the similarity between the original and reconstructed data can tell you how consistent and efficient the mappings are.

In [ ]:
# A -> B

# Define transformation parameters
angle_deg = 30.6       # Rotation angle in degrees
tx = 22.3              # Translation along x
ty = 10.15             # Translation along y

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)

T = np.array([
    [cos_a, -sin_a, tx],
    [sin_a, cos_a, ty],
    [0,0,1]
]) 

# Construct image
output_AtoB = out_backward_bilinear(image, T)
output_AtoB_2 = out_backward_nn(image, T)

In [ ]:
# A -> B

# Define transformation parameters
angle_deg = 30.6       # Rotation angle in degrees
tx = 22.3              # Translation along x
ty = 10.15             # Translation along y

# Convert angle to radians
angle_rad = np.deg2rad(angle_deg)

# Calculate cosine and sine of angle
cos_a = np.cos(angle_rad)
sin_a = np.sin(angle_rad)

T = np.array([
    [cos_a, -sin_a, tx],
    [sin_a, cos_a, ty],
    [0,0,1]
]) 

# Construct image
output_BtoA = forward_bilinear(output_AtoB, T)
output_BtoA_2 = out_forward_nn(output_AtoB_2, T2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 8))

# Show the original image
axes[0].imshow(image, cmap='gray')
axes[0].set_title("Original Image")
axes[0].axis("off")

# Show the AtoBtoA image (Bil)
axes[1].imshow(output_BtoA, cmap='gray')
axes[1].set_title("A->B->A Mapping (Bil operator)")
axes[1].axis("off")

# Show the AtoBtoA image (NN)
axes[2].imshow(output_BtoA_2, cmap='gray')
axes[2].set_title("A->B->A Mapping (NN operator)")
axes[2].axis("off")

In [ ]:
# Measure differences
mse_valAtoBtoA = mse(output_BtoA, image)
psnr_valAtoBtoA = psnr(output_BtoA, image)
ssim_valAtoBtoA = ssim(output_BtoA, image)

mse_valAtoBtoA_NN = mse(output_BtoA_2, image)
psnr_valAtoBtoA_NN = psnr(output_BtoA_2, image)
ssim_valAtoBtoA_NN = ssim(output_BtoA_2, image)

    
#Given
print(f"{'Method':<20} | {'MSE':>10} | {'PSNR':>10} | {'SSIM':>10}")
print("-" * 50)
print(f"{'AtoBtoA':<20} | {mse_valAtoBtoA:10.2f} | {psnr_valAtoBtoA:10.2f} | {ssim_valAtoBtoA:10.4f}")
print(f"{'AtoBtoA NN':<20} | {mse_valAtoBtoA_NN:10.2f} | {psnr_valAtoBtoA_NN:10.2f} | {ssim_valAtoBtoA_NN:10.4f}")

Nous obtenons un résultat très concluant pour la méthode avec l'opérateur bilinéaire, permettant ainsi de combler les trou grâce à l'interpolation et la pondération des pixels voisins pour les pixels manquants. Nous observons un MSE très important, du fait de la perte d'information lors des transformation (des pixels (noirs) hors du cadre de l'image de base se sont retrouver dans le cadre de l'image lors de la première transformation, ce qui est une perte d'information non-négligeable, impossible à rattraper lors de la deuxième opération).
Les résultats pour la méthode avec l'opérateur NN sont quand à eux très faible. Nous pouvons cependant distinguer très nettement l'objet de départ (parrot), les algorithmes sont très simple à implémenter et ne nécessitent pas une grosse capacités de calculs. Pour des algorithmes demandant beaucoup de calculs (classification avec beaucoup d'epoch, beaucoup d'individus) cette méthode pourrait suffire.

* **Your Feedback (Optional but highly recommended for improving the BE session)**

  1. Was the session too long or too short?
  2. Did you find it easy or difficult?
  3. Was it engaging or uninteresting?
  4. Did you feel adequately guided throughout the session?
  5. What aspects could be improved?"

----------------

La session était clairement trop courte pour faire le BE en entier, et ce BE nous a pris beaucoup plus de temps que les précédents. Il n'était pas compliqué, plutôt bien guidé, et basé sur les notions vu en cours. Le problème est qu'il est très répétitif, avec toujours les mêmes questions qui nous oblige à se répéter. L'expériment 5 est nettement plus intéressant, et permet de mieux visualiser les opération (et combinaison d'opération) que l'on effectue. Un aspect qui pourrait être amélioré serait de directement guider les étudiants à l'élaboration des 4 méthodes (forward NN / Bil et backward NN / Bil) afin de visualiser plus en détails les différences et d'analyser qu'une seule fois ces différences